# 🌟 00: LangGraph 기본 개념 맛보기

## 💡 5분만에 이해하는 3대 핵심 개념!

LangGraph의 핵심은 **State**(상태), **Node**(노드), **Graph**(그래프) 세 가지입니다.  
이 노트북에서는 복잡한 이론 없이 실습으로 빠르게 체험해봅니다.

## 🎯 이 노트북에서 배우는 것들

### ✅ 핵심 학습 목표
1. **State (상태)**: 워크플로우에서 데이터를 저장하고 공유하는 방법
2. **Node (노드)**: State를 처리하는 함수의 구조와 역할
3. **Graph (그래프)**: 노드들을 연결해서 워크플로우 만드는 방법

### 💡 학습 비중
각 개념을 **균형있게** 배웁니다:
- State: 데이터 흐름과 업데이트 방식 이해
- Node: 처리 로직을 독립적인 함수로 만들기
- Graph: 노드들을 연결해 실행 순서 정의

### ⏱️ 예상 소요시간: 5분

### 📚 다음 학습
- `01-advanced-integration.ipynb`에서 Annotated, Command, interrupt 등 고급 패턴 학습

In [ ]:
# 필요한 라이브러리 설치
!pip install -q langgraph langchain langchain-teddynote
!pip install -q grandalf matplotlib networkx pyppeteer

# 기본 import
from typing import TypedDict
from langgraph.graph import StateGraph, END, START
from langchain_teddynote.graphs import visualize_graph

print("✅ LangGraph 준비 완료!")

## 🎯 State란 뭘까?

**State = 워크플로우에서 노드들이 공유하는 데이터 저장소**

### 핵심 개념:
1. **공유 저장소**: 모든 노드가 State를 읽고 업데이트할 수 있음
2. **부분 업데이트**: 필요한 필드만 업데이트하면, 나머지는 자동으로 유지됨
3. **타입 정의**: `TypedDict`로 어떤 데이터가 들어가는지 명확히 정의

### State의 2가지 업데이트 방식:
- **기본 (덮어쓰기)**: 마지막 값으로 교체
- **Annotated (누적)**: 리스트 추가, 숫자 합계 등 (01번에서 배움)

In [ ]:
# 가장 간단한 State 정의
class SimpleState(TypedDict):
    input: str      # 사용자가 넣을 데이터
    output: str     # 최종 결과가 나올 곳

# 예시 State
example_state = {
    "input": "안녕하세요",
    "output": ""  # 아직 빈 상태
}

print("🗂️ SimpleState 정의 완료!")
print(f"📝 예시: {example_state}")

print("\n💡 핵심 포인트:")
print("   - State는 딕셔너리처럼 생겼지만, 타입이 정해져 있어요")
print("   - 노드들이 이 State를 읽고 → 처리하고 → 업데이트합니다")
print("   - 필요한 필드만 업데이트해도 나머지는 자동으로 유지돼요!")

## 🔧 Node란 뭘까?

**Node = State를 받아서 처리하고 업데이트하는 함수**

### Node의 3가지 규칙:
1. **State를 파라미터로 받음**: `def my_node(state: SimpleState)`
2. **필요한 필드만 업데이트**: 전체 State를 다시 쓸 필요 없음!
3. **딕셔너리 리턴**: 업데이트할 필드만 포함

### 💡 핵심 원리:
```
❌ Bad: 모든 필드를 다시 써야 함
   return {"input": state["input"], "output": "결과"}  # input 왜 다시 써?

✅ Good: 필요한 것만 업데이트
   return {"output": "결과"}  # input은 자동 유지!
```

In [ ]:
# 가장 간단한 Node 함수
def hello_node(state: SimpleState) -> SimpleState:
    """인사해주는 노드
    
    ✅ 좋은 예: output만 업데이트
    - input은 자동으로 유지됨
    - 전체 State를 다시 쓸 필요 없음
    """
    user_input = state["input"]
    greeting = f"🤖 안녕하세요, '{user_input}'라고 말씀하셨군요!"
    
    # 핵심: output만 리턴! input은 자동 유지됨
    return {"output": greeting}

# 테스트해보기
test_state = {"input": "반가워요", "output": ""}
result_state = hello_node(test_state)

print("✅ hello_node 함수 정의 완료!")
print(f"🧪 테스트 입력: {test_state}")
print(f"🧪 테스트 출력: {result_state}")

print("\n💡 관찰:")
print("   - 리턴값에 'input'이 없지만, 결과에는 있어요!")
print("   - LangGraph가 자동으로 기존 값을 유지합니다")

## 🚀 Graph로 노드 연결하기

이제 **StateGraph**로 노드를 연결해서 워크플로우를 만들어보자!

### Graph의 4단계:
1. `StateGraph(State)`: 그래프 생성
2. `add_node()`: 노드 추가
3. `add_edge()`: 실행 순서 정의
4. `compile()`: 실행 가능한 앱으로 만들기

In [ ]:
# 1개 노드로 구성된 가장 간단한 워크플로우
workflow = StateGraph(SimpleState)

# 노드 추가
workflow.add_node("hello", hello_node)

# 시작점과 끝점 설정
workflow.set_entry_point("hello")
workflow.add_edge("hello", END)

# 실행 가능한 앱으로 컴파일
app = workflow.compile()

print("🎯 첫 번째 워크플로우 완성!")
print("   흐름: START → hello → END")
print("\n💡 핵심:")
print("   - add_node()로 노드 등록")
print("   - add_edge()로 실행 순서 정의")
print("   - compile()로 실행 가능한 상태로 만들기")

In [ ]:
# 그래프 시각화
print("📊 워크플로우 구조:")
print("   - 박스(노드): State를 처리하는 함수")
print("   - 화살표(엣지): 실행 순서")
print("   - __start__: 진입점 / __end__: 종료점")
print()
visualize_graph(app)

In [ ]:
# 워크플로우 실행해보기
initial_state = {
    "input": "LangGraph 배우는 중!",
    "output": ""
}

print("🚀 워크플로우 실행:")
print(f"📥 초기: {initial_state}")

final_state = app.invoke(initial_state)
print(f"📤 최종: {final_state}")
print(f"\n🎉 결과: {final_state['output']}")

## 🔗 노드 여러 개 연결하기

노드를 3개 만들어서 순서대로 실행해보자!  
hello → process → format → 끝

In [ ]:
# 추가 노드들 정의
def process_node(state: SimpleState) -> SimpleState:
    """입력을 처리하는 노드"""
    user_input = state["input"]
    processed = f"⚙️ '{user_input}'을(를) 처리했습니다"
    return {"output": processed}

def format_node(state: SimpleState) -> SimpleState:
    """결과를 예쁘게 포맷하는 노드"""
    current_output = state["output"]
    formatted = f"✨ [최종결과] {current_output} ✨"
    return {"output": formatted}

print("✅ 노드 2개 추가 정의 완료!")

In [ ]:
# 3개 노드를 연결한 워크플로우
multi_workflow = StateGraph(SimpleState)

# 노드들 추가
multi_workflow.add_node("hello", hello_node)
multi_workflow.add_node("process", process_node)
multi_workflow.add_node("format", format_node)

# 연결: hello → process → format → END
multi_workflow.set_entry_point("hello")
multi_workflow.add_edge("hello", "process")
multi_workflow.add_edge("process", "format")
multi_workflow.add_edge("format", END)

# 컴파일
multi_app = multi_workflow.compile()

print("🔗 3단계 워크플로우 완성!")
print("   흐름: hello → process → format → END")

In [ ]:
# 3단계 워크플로우 실행
test_input = {
    "input": "노드 체인 테스트",
    "output": ""
}

print("🧪 3단계 워크플로우 실행:")
print(f"📥 입력: '{test_input['input']}'")

final_result = multi_app.invoke(test_input)

print(f"📤 결과: {final_result['output']}")
print("\n💡 관찰: 3개 노드가 순서대로 State를 업데이트했어요!")

## 🔬 State 변화 추적하기 (중요!)

지금까지는 `app.invoke()`로 워크플로우를 실행했어요. 그러면 **최종 결과만** 보입니다.

하지만 "각 노드가 State를 어떻게 바꾸는지" 중간 과정을 보려면?  
→ 노드를 **직접 호출**하고 **수동으로 병합**하면 됩니다!

### 💡 핵심 개념: 파이썬 딕셔너리 병합

```python
dict1 = {"a": 1, "b": 2}
dict2 = {"b": 3, "c": 4}  # b는 덮어쓰고, c는 새로 추가

merged = {**dict1, **dict2}
# 결과: {"a": 1, "b": 3, "c": 4}
#       ↑ 유지  ↑ 덮어씀  ↑ 추가
```

**이게 바로 LangGraph가 내부적으로 하는 일입니다!**

### 🎯 아래 셀의 목적:
- 노드를 직접 호출 → 각 노드가 뭘 리턴하는지 확인
- `{**현재State, **노드리턴값}` 으로 병합 → State가 어떻게 변하는지 추적
- **교육용 시뮬레이션**: 실제로는 `app.invoke()`가 자동으로 해줌

In [ ]:
# State 변화 추적 실습
print("🔬 각 노드가 State를 어떻게 변경하는지 단계별 확인:")
print("=" * 60)

# 헬퍼 함수: 딕셔너리 병합 (LangGraph 내부 동작 재현)
def merge_state(current_state, node_update):
    """
    {**dict1, **dict2} = 파이썬 딕셔너리 언패킹
    - dict1의 모든 키 복사
    - dict2의 키로 덮어쓰기/추가
    - dict1에만 있는 키는 자동 유지!
    """
    return {**current_state, **node_update}

initial = {"input": "단계별 추적", "output": ""}
print(f"🏁 초기 State: {initial}")
print()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 1: hello_node 실행
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
hello_update = hello_node(initial)  # 노드를 직접 호출!
after_hello = merge_state(initial, hello_update)  # 수동 병합

print("1️⃣ hello_node 실행:")
print(f"   노드 리턴값: {hello_update}")
print(f"   👆 'input' 필드가 없어요! output만 리턴")
print(f"   병합 결과: {after_hello}")
print(f"   👆 'input'은 initial에서 자동 유지됨!")
print()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 2: process_node 실행
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
process_update = process_node(after_hello)
after_process = merge_state(after_hello, process_update)

print("2️⃣ process_node 실행:")
print(f"   노드 리턴값: {process_update}")
print(f"   병합 결과 output: '{after_process['output'][:40]}...'")
print()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# Step 3: format_node 실행
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
format_update = format_node(after_process)
after_format = merge_state(after_process, format_update)

print("3️⃣ format_node 실행:")
print(f"   노드 리턴값: {format_update}")
print(f"   병합 결과 output: '{after_format['output'][:45]}...'")
print()

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# 핵심 정리
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("💡 핵심 발견:")
print("   ✅ 각 노드는 필요한 필드(output)만 리턴")
print("   ✅ {**current, **update}로 자동 병합")
print("   ✅ 'input' 필드는 한 번도 리턴 안 했지만 계속 유지!")
print()
print("🎯 실제 LangGraph:")
print("   - app.invoke()를 쓰면 이 병합 과정을 자동으로 해줌")
print("   - 여기서는 교육 목적으로 수동 시뮬레이션한 것!")

## 🤔 조건부 분기 맛보기

입력에 따라 다른 경로로 처리하는 방법을 배워봅시다!  
- 짧은 입력 → 간단 처리
- 긴 입력 → 복잡 처리

In [ ]:
# 조건부 분기를 위한 상태와 함수들
class BranchState(TypedDict):
    input: str
    output: str
    path: str  # 어떤 경로로 처리됐는지 추적

def decide_path(state: BranchState):
    """입력 길이에 따라 경로 결정
    
    💡 실무에서는:
    - LLM이 문의 유형 판단 (기술/영업/환불 등)
    - 임베딩 유사도, 규칙 기반 분류 등 사용
    - 여기서는 학습용으로 단순한 길이 체크만 사용
    """
    input_text = state["input"]
    if len(input_text) <= 5:
        return "simple"
    else:
        return "complex"

def simple_process(state: BranchState):
    """짧은 입력용 간단 처리"""
    return {
        "output": f"🟢 간단: '{state['input']}' → 빠른 응답",
        "path": "simple"
    }

def complex_process(state: BranchState):
    """긴 입력용 복잡 처리"""
    return {
        "output": f"🔵 복잡: '{state['input']}' → 상세 분석",
        "path": "complex"
    }

print("✅ 조건부 분기용 함수들 정의 완료!")

In [ ]:
# 조건부 분기 워크플로우
branch_workflow = StateGraph(BranchState)

# 노드들 추가
branch_workflow.add_node("simple", simple_process)
branch_workflow.add_node("complex", complex_process)

# 시작점은 조건부 엣지
branch_workflow.set_conditional_entry_point(
    decide_path,  # 판단 함수
    {
        "simple": "simple",    # 짧으면 → simple 노드
        "complex": "complex"   # 길면 → complex 노드
    }
)

# 둘 다 END로
branch_workflow.add_edge("simple", END)
branch_workflow.add_edge("complex", END)

# 컴파일
branch_app = branch_workflow.compile()

print("🌿 조건부 분기 워크플로우 완성!")
print("   흐름: START → [길이판단] → simple/complex → END")

In [ ]:
# 조건부 분기 워크플로우 시각화
print("📊 조건부 분기 그래프:")
visualize_graph(branch_app)

In [ ]:
# 조건부 분기 동작 테스트
test_cases = [
    ("안녕", "2글자 → simple"),
    ("안녕하세요!", "6글자 → complex"),
    ("LangGraph 최고", "12글자 → complex")
]

print("🧪 조건부 분기 테스트:")
print("📏 기준: 5글자 이하 → simple, 6글자 이상 → complex")
print("=" * 60)

for test_input, expected in test_cases:
    result = branch_app.invoke({
        "input": test_input,
        "output": "",
        "path": ""
    })
    
    print(f"\n입력: '{test_input}' ({len(test_input)}글자)")
    print(f"경로: {result['path']} ✅")

print("\n🎓 조건부 분기로 입력에 따라 다른 처리를 할 수 있어요!")

## 💼 실무 활용 예시

지금까지 배운 **State + Node + Graph + 조건부 분기**를 실무에 적용하면:

### 고객 문의 자동 라우팅
```
State: {message: "환불 문의", category: "", team: ""}
    ↓
[classify_node] → 카테고리 분류
    ↓ (조건부 분기)
    ├─ 환불팀 → [refund_handler]
    ├─ 배송팀 → [delivery_handler]
    └─ 기술팀 → [tech_handler]
```

### AI 에이전트 (다음 노트북에서!)
```
State: {query: "서울 날씨", answer: ""}
    ↓
[think_node] → 도구 선택 (LLM 판단)
    ↓ (조건부 분기)
    ├─ 웹 검색 필요 → [search_tool]
    ├─ 계산 필요 → [calculator_tool]
    └─ 충분 → [answer_node]
```

**💡 핵심:** 이 노트북의 패턴 = 실무 AI 시스템의 뼈대!

## 🎓 학습 완료! 기본 개념 정리

### ✅ 지금까지 배운 것 (3대 핵심 개념)

#### 1. State (상태) 📦
- 워크플로우에서 노드들이 공유하는 데이터 저장소
- `TypedDict`로 명확한 타입 정의
- **부분 업데이트**: 필요한 필드만 업데이트, 나머지는 자동 유지
- 업데이트 방식: 기본(덮어쓰기) / Annotated(누적)

#### 2. Node (노드) ⚙️
- State를 처리하는 함수
- 3가지 규칙:
  1. State를 파라미터로 받음
  2. 필요한 필드만 업데이트
  3. 딕셔너리 리턴
- 독립적으로 테스트 가능

#### 3. Graph (그래프) 🔗
- 노드들을 연결한 워크플로우
- `add_node()`: 노드 등록
- `add_edge()`: 실행 순서 정의
- `compile()`: 실행 가능한 앱 생성
- `invoke()`: 워크플로우 실행
- **조건부 분기**: `set_conditional_entry_point()`로 동적 라우팅

### 📊 비중
이 노트북에서 3가지 개념을 균형있게 배웠습니다:
- State 33% + Node 33% + Graph 33% = 완벽한 기초!

### 🚀 다음 학습 단계

**본격 실습 (main-practice):**
- `main-practice/00-01-langgraph-fundamentals-guide.ipynb`
  - LLM을 노드로 사용
  - MemorySaver로 대화 기억
  - interrupt()로 Human-in-the-Loop

**선택적 심화 (prerequisites):**
- `01-advanced-integration.ipynb`: Annotated, Command, interrupt, Fan-in 등 고급 패턴
- `05-multi-agent-handoffs.ipynb`: 여러 에이전트 협업

### ⚠️ 실무 체크리스트 (참고용)
나중에 프로덕션 배포 시:
- ✅ 에러 처리: 노드에 `try-except` 추가
- ✅ State 검증: Pydantic으로 타입 강제
- ✅ 로깅: 각 노드 입출력 기록
- ✅ 모니터링: LangSmith로 실행 추적

### 🎉 축하합니다!
LangGraph의 기본 개념을 마스터했어요!  
이제 main-practice로 넘어가서 본격적인 AI 에이전트를 만들어봅시다! 🚀